In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison

import copy
import seaborn as sns
import os
from scipy import stats
import datetime

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [4]:
path='/Users/carlyfennell/DataScience/data_bootcamp/data/'

# Skip all this - already did once

# Feature Engineering

## finding latitude and longitude for all rows

## join lat/long for all rows of df_flights table based on origin

# optimizing datatypes in dataframe

https://www.dataquest.io/blog/pandas-big-data/

In [54]:
df_flights=pd.read_csv(path+'flights.csv')

/opt/anaconda3/envs/bootcamp_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Strange fix to add missing lat/long to table

# Start Here

## Read in data from saved .csv dataframes

In [36]:
column_types={'mkt_carrier_fl_num': 'uint16', 'op_unique_carrier': 'category', 'tail_num': 'category', 'op_carrier_fl_num': 'uint16', 'origin_airport_id': 'uint16', 'origin': 'category', 'dest_airport_id': 'uint16', 'dest': 'category', 'crs_dep_time': 'uint16', 'dep_time': 'float32'}

In [39]:
flight_A=pd.read_csv(path+'flights_with_weather_first.csv',dtype=column_types,parse_dates=['date'],infer_datetime_format=True)

In [41]:
flight_A=flight_A.drop('Unnamed: 0',axis=1)

In [55]:
flight_A.dtypes

mkt_unique_carrier      object
branded_code_share      object
mkt_carrier_fl_num      uint16
tail_num              category
origin                category
dest                  category
crs_dep_time            uint16
dep_delay              float64
crs_arr_time             int64
arr_time               float64
arr_delay              float64
distance                 int64
year                     int64
month                    int64
date                    object
latitude               float64
longitude              float64
wdir                   float64
period                  object
temp                   float64
maxt                   float64
visibility             float64
wspd                   float64
heatindex              float64
cloudcover             float64
mint                   float64
precip                 float64
snowdepth              float64
sealevelpressure       float64
dew                    float64
humidity               float64
wgust                  float64
precipco

In [53]:
pd.DataFrame(flight_A.dtypes[(flight_A.dtypes=='object')|(flight_A.dtypes=='category')]).reset_index().rename(columns={'index':'Variable',0:'data_type'})

,Variable,data_type
0,mkt_unique_carrier,object
1,branded_code_share,object
2,tail_num,category
3,origin,category
4,dest,category
5,date,object
6,period,object
7,closest_airport,object


# Weather

In [5]:
#read in weather info Dataframe
A_W=pd.read_csv(path+'agg_weath_10_26.csv')
A_W=A_W.drop('Unnamed: 0',axis=1)

In [86]:
A_W[['latitude','longitude']].iloc[10:14]

,latitude,longitude
10,26.919414,-81.993028
11,26.919414,-81.993028
12,29.647149,-95.276926
13,29.647149,-95.276926



explanation for lat/lon closest to point code

https://stackoverflow.com/questions/41336756/find-the-closest-latitude-and-longitude

In [13]:
opt_flights['month'].unique()

array([ 5,  2,  9,  4,  7,  3,  1,  8, 10, 11,  6, 12])

# Calculating nearest airport based on lat/lon

In [13]:
from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    return min(data, key=lambda p: distance(v['lat'],v['lon'],p['lat'],p['lon']))

In [5]:
#build dictionary tempDataList from path+'weather_dict.csv'
tempDataList_df=pd.read_csv(path+'weather_dict.csv')

In [8]:
tempDataList_df.columns

Index(['Unnamed: 0', 'lat', 'lon', 'org'], dtype='object')

In [10]:
tempDataList_df=tempDataList_df.drop('Unnamed: 0',axis=1)

In [20]:
tempDataList=list(tempDataList_df.T.to_dict().values())

In [21]:
tempDataList

[{'lat': 26.9194137, 'lon': -81.9930281031229, 'org': 'PGD'},
 {'lat': 29.64714905, 'lon': -95.27692608634595, 'org': 'HOU'},
 {'lat': 38.95226625, 'lon': -77.45342297783296, 'org': 'IAD'},
 {'lat': 43.590338, 'lon': -79.645729, 'org': 'ONT'},
 {'lat': 20.89527825, 'lon': -156.4328644795689, 'org': 'OGG'},
 {'lat': 59.8474657, 'lon': 17.63105350585749, 'org': 'SGU'},
 {'lat': 49.88527, 'lon': 2.80813, 'org': 'FAY'},
 {'lat': 52.52183095, 'lon': 13.363127119018849, 'org': 'BMI'},
 {'lat': 36.638392, 'lon': 127.6961188, 'org': 'SAN'},
 {'lat': 46.56681015, 'lon': -120.53679819122014, 'org': 'YKM'},
 {'lat': 42.4036084, 'lon': -96.3836488, 'org': 'SUX'},
 {'lat': 43.2768745, 'lon': 0.092596, 'org': 'BUR'},
 {'lat': 41.31116815, 'lon': -8.453010682764061, 'org': 'STS'},
 {'lat': 45.6705089, 'lon': 24.1822969, 'org': 'COD'},
 {'lat': 15.510172400000002, 'lon': -12.850290199999998, 'org': 'MGM'},
 {'lat': 26.8106778, 'lon': 80.89093790000003, 'org': 'MKG'},
 {'lat': 12.91127585, 'lon': 77.60

In [22]:
print(closest(tempDataList,{'lat': 39.7622290, 'lon': -86.1519750}))

{'lat': 43.590338, 'lon': -79.645729, 'org': 'ONT'}


In [37]:
#### THIS TEST SEEMS TO BE WORKING
#for i in range(test_df):
for i in range(2):
    print(closest(tempDataList,{'lat':opt_flights['ori_latitude'][i],'lon':opt_flights['ori_longitude'][i]}))
    

{'lat': 33.94216754999999, 'lon': -118.42137629872809, 'org': 'LAX'}
{'lat': 33.94216754999999, 'lon': -118.42137629872809, 'org': 'LAX'}


In [30]:
aw_vars=['wdir', 'period', 'temp', 'maxt', 'visibility', 'wspd',
       'heatindex', 'cloudcover', 'mint', 'precip', 'snowdepth',
       'sealevelpressure', 'dew', 'humidity', 'wgust', 'precipcover',
       'windchill']

In [31]:
ori_vars=['wdir_ori', 'period_ori', 'temp_ori', 'maxt_ori', 'visibility_ori',
       'wspd_ori', 'heatindex_ori', 'cloudcover_ori', 'mint_ori', 'precip_ori',
       'snowdepth_ori', 'sealevelpressure_ori', 'dew_ori', 'humidity_ori',
       'wgust_ori', 'precipcover_ori', 'windchill_ori']

In [32]:
dest_vars=['wdir_dest',
       'period_dest', 'temp_dest', 'maxt_dest', 'visibility_dest', 'wspd_dest',
       'heatindex_dest', 'cloudcover_dest', 'mint_dest', 'precip_dest',
       'snowdepth_dest', 'sealevelpressure_dest', 'dew_dest', 'humidity_dest',
       'wgust_dest', 'precipcover_dest', 'windchill_dest']

In [82]:
A_W.loc[(A_W['origin']=='LAX')&(A_W['month']==1)][aw_vars]
#opt_flights.loc[(opt_flights['origin']=='LAX') & (opt_flights['month']==1), ori_vars]

,wdir,period,temp,maxt,visibility,wspd,heatindex,cloudcover,mint,precip,snowdepth,sealevelpressure,dew,humidity,wgust,precipcover,windchill
216,155.18,Jan,59.3,87.8,8.7,31.4,84.2,45.4,40.0,6.98,NaN,1018.8,44.4,64.04,40.0,7.4,35.2


# Create Dataframe key with 376 unique airports, and the month 

In [23]:
#read weather_key into dataframe
weather_key=pd.read_csv(path+'wkey.csv')
weather_key=weather_key.drop(['Unnamed: 0','period'],axis=1)

# try merge

In [169]:
weather_key.columns

Index(['origin', 'month', 'latitude', 'longitude', 'wdir', 'period', 'temp',
       'maxt', 'visibility', 'wspd', 'heatindex', 'cloudcover', 'mint',
       'precip', 'snowdepth', 'sealevelpressure', 'dew', 'humidity', 'wgust',
       'precipcover', 'windchill', 'closest_airport'],
      dtype='object')

In [171]:
opt_flights.columns

Index(['mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'dest_airport_id',
       'dest', 'crs_dep_time', 'dep_time', 'dep_delay', 'taxi_out',
       'wheels_off', 'wheels_on', 'taxi_in', 'crs_arr_time', 'arr_time',
       'arr_delay', 'cancelled', 'cancellation_code', 'diverted',
       'crs_elapsed_time', 'actual_elapsed_time', 'air_time', 'flights',
       'distance', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay', 'first_dep_time',
       'total_add_gtime', 'longest_add_gtime', 'no_name', 'ori_latitude',
       'ori_longitude', 'dest_latitude', 'dest_longitude', 'fl_date', 'month'],
      dtype='object')

In [172]:
opt_flights.shape

(15927485, 41)

In [174]:
#new_df = pd.merge(A_df, B_df,  how='left', left_on=['A_c1','c2'], right_on = ['B_c1','c2'])
new_df=pd.merge(opt_flights,weather_key[['origin','month', 'wdir', 'temp',
       'maxt', 'visibility', 'wspd', 'heatindex', 'cloudcover', 'mint',
       'precip', 'snowdepth', 'sealevelpressure', 'dew', 'humidity', 'wgust',
       'precipcover', 'windchill', 'closest_airport']],how='outer',on=['origin','month'])

In [33]:
test_keys = aw_vars
test_values = ori_vars
test_values2=dest_vars
res = dict(zip(test_keys, test_values)) 
res2=dict(zip(test_keys, test_values2)) 

In [34]:
res

{'wdir': 'wdir_ori',
 'period': 'period_ori',
 'temp': 'temp_ori',
 'maxt': 'maxt_ori',
 'visibility': 'visibility_ori',
 'wspd': 'wspd_ori',
 'heatindex': 'heatindex_ori',
 'cloudcover': 'cloudcover_ori',
 'mint': 'mint_ori',
 'precip': 'precip_ori',
 'snowdepth': 'snowdepth_ori',
 'sealevelpressure': 'sealevelpressure_ori',
 'dew': 'dew_ori',
 'humidity': 'humidity_ori',
 'wgust': 'wgust_ori',
 'precipcover': 'precipcover_ori',
 'windchill': 'windchill_ori'}

In [35]:
res2

{'wdir': 'wdir_dest',
 'period': 'period_dest',
 'temp': 'temp_dest',
 'maxt': 'maxt_dest',
 'visibility': 'visibility_dest',
 'wspd': 'wspd_dest',
 'heatindex': 'heatindex_dest',
 'cloudcover': 'cloudcover_dest',
 'mint': 'mint_dest',
 'precip': 'precip_dest',
 'snowdepth': 'snowdepth_dest',
 'sealevelpressure': 'sealevelpressure_dest',
 'dew': 'dew_dest',
 'humidity': 'humidity_dest',
 'wgust': 'wgust_dest',
 'precipcover': 'precipcover_dest',
 'windchill': 'windchill_dest'}

In [188]:
new_df=new_df.rename(columns=res)

In [189]:
new_df.columns

Index(['mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'dest_airport_id',
       'dest', 'crs_dep_time', 'dep_time', 'dep_delay', 'taxi_out',
       'wheels_off', 'wheels_on', 'taxi_in', 'crs_arr_time', 'arr_time',
       'arr_delay', 'cancelled', 'cancellation_code', 'diverted',
       'crs_elapsed_time', 'actual_elapsed_time', 'air_time', 'flights',
       'distance', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay', 'first_dep_time',
       'total_add_gtime', 'longest_add_gtime', 'no_name', 'ori_latitude',
       'ori_longitude', 'dest_latitude', 'dest_longitude', 'fl_date', 'month',
       'wdir_ori', 'temp_ori', 'maxt_ori', 'visibility_ori', 'wspd_ori',
       'heatindex_ori', 'cloudcover_ori', 'mint_ori', 'precip_ori',
       'snowdepth_ori', 'sealevelpressure_ori', 'dew_ori', 'humidity_ori',
       'wgust_ori', 'precipcover_ori', 'windchill_ori', 'closest_airport'],
   

In [ ]:
new_df_two=pd.merge(new_df,weather_key[['origin','month', 'wdir', 'temp',
       'maxt', 'visibility', 'wspd', 'heatindex', 'cloudcover', 'mint',
       'precip', 'snowdepth', 'sealevelpressure', 'dew', 'humidity', 'wgust',
       'precipcover', 'windchill', 'closest_airport']],how='outer',left_on=['dest','month'],right_on=['origin','month'])

In [191]:
new_df_two.columns

Index(['mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin_x', 'dest_airport_id',
       'dest', 'crs_dep_time', 'dep_time', 'dep_delay', 'taxi_out',
       'wheels_off', 'wheels_on', 'taxi_in', 'crs_arr_time', 'arr_time',
       'arr_delay', 'cancelled', 'cancellation_code', 'diverted',
       'crs_elapsed_time', 'actual_elapsed_time', 'air_time', 'flights',
       'distance', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay', 'first_dep_time',
       'total_add_gtime', 'longest_add_gtime', 'no_name', 'ori_latitude',
       'ori_longitude', 'dest_latitude', 'dest_longitude', 'fl_date', 'month',
       'origin_y', 'wdir', 'temp', 'maxt', 'visibility', 'wspd', 'heatindex',
       'cloudcover', 'mint', 'precip', 'snowdepth', 'sealevelpressure', 'dew',
       'humidity', 'wgust', 'precipcover', 'windchill', 'closest_airport'],
      dtype='object')